# **Notebook 03**

Todos os scripts usam a sintaxe T-SQL.

Os resultados da consultas foram limitados a um número específico de linhas, para facilitar a visualização do notebook

<br>

* * *

## **Desafio**

### O desafio consiste em atualizar os nomes de categorias de produto para uma versão de melhor leitura (Ex: moveis\_decoracao \> Movéis e Decoração) nos registros da tabela _olist\_products_ que têm altura maior ou igual a 20cm e cujo tamanho do nome esteja entre 10 e 200 caracteres.

Primeiro, vamos identificar os nomes distintos dos produtos que atendem ao critério da consulta (altura maior ou igual a 20cm e nome entre 10 e 200 caracteres):

In [35]:
SELECT 
    DISTINCT(product_category_name) 
FROM olist_products
WHERE 
    product_height_cm >= 20 
    AND product_name_length BETWEEN 10 and 200;

(71 rows affected)

Total execution time: 00:00:00.208

product_category_name
casa_conforto_2
perfumaria
pcs
fashion_calcados
artigos_de_natal
bebidas
construcao_ferramentas_jardim
construcao_ferramentas_seguranca
telefonia
automotivo


Pela tabela acima, existem 71 categorias únicas que correspondem aos produtos que atendem aos critérios da consulta, sem qualquer padrão de nomenclatura que nos permita alcançar o resultado esperado com uma manipulação simples de texto. Assim, a melhor estratégia parece ser a de criarmos uma tabela que mapeie os nomes atuais das categorias e suas versões "melhoradas":

In [36]:
/* Como o T-SQL não dispõe de funções simples de split e capitalização do primeiro caracter de uma string,
 * o caminho mais seguro parece ser o de mapear manualmente as categorias acima para versões melhores.
 * Para isso, vamos criar uma tabela temporária para mapear os nomes de categorias:
 */
CREATE TABLE #map_product_category_names (
    ugly_name NVARCHAR(255),
    pretty_name NVARCHAR(255)
);

INSERT INTO #map_product_category_names (ugly_name, pretty_name)
VALUES  
    ('casa_conforto_2', 'Casa e Conforto 2'),   ('perfumaria', 'Perfumaria'),
    ('pcs', 'PCs'),     ('fashion_calcados', 'Fashion / Calçados'),     ('artigos_de_natal', 'Artigos de Natal'),
    ('bebidas', 'Bebidas'),     ('construcao_ferramentas_jardim', 'Construção e Ferramentas / Jardim'),     
    ('construcao_ferramentas_seguranca', 'Consturção e Ferramentas / Segurança'),   ('telefonia', 'Telefonia'),     
    ('automotivo', 'Automotivo'),       ('flores', 'Flores'),   
    ('industria_comercio_e_negocios', 'Indústria, Comércio e Negócios'),    
    ('fashion_bolsas_e_acessorios', 'Fashion / Bolsas e Acessórios'),   ('beleza_saude', 'Beleza e Saúde'),
    ('telefonia_fixa', 'Telefonia Fixa'),   ('alimentos_bebidas', 'Alimentos e Bebidas'),   
    ('dvds_blu_ray', 'DVDs e Blu Rays'),    ('artes', 'Artes'), ('eletronicos', 'Eletrônicos'), 
    ('moveis_sala', 'Móveis / Sala'),   ('brinquedos', 'Brinquedos'),   
    ('sinalizacao_e_seguranca', 'Sinalização e Segurança'), ('utilidades_domesticas', 'Utilidades Domésticas'),
    ('fashion_roupa_feminina', 'Fashion / Roupa Feminina'), 
    ('fashion_roupa_infanto_juvenil', 'Fasion / Roupa Infantojuvenil'), 
    ('agro_industria_e_comercio', 'Agro, Indústria e Comércio'),    ('cine_foto', 'Cine e Foto'),   
    ('livros_tecnicos', 'Livros Técnicos'), ('fashion_esporte', 'Fashio / Esportes'),
    ('construcao_ferramentas_iluminacao', 'Construção e Ferramentas / Iluminação'), ('moveis_quarto', 'Móveis / Quarto'),   
    ('portateis_casa_forno_e_cafe', 'Portáteis / Casa / Forno e Café'), ('eletrodomesticos_2', 'Eletrodomésticos 2'),   
    ('livros_interesse_geral', 'Livros de Interesse Geral'),    ('pc_gamer', 'PC Gamer'),
    ('relogios_presentes', 'Relógios e Presentes'),     
    ('portateis_cozinha_e_preparadores_de_alimentos', 'Portáteis / Cozinha e Preparadores de Alimentos'),   
    ('audio', 'Áudio'), ('alimentos', 'Alimentos'), 
    ('construcao_ferramentas_ferramentas', 'Construção e Ferramentas / Ferramentas'),
    ('climatizacao', 'Climatização'),   ('pet_shop', 'Pet Shop'),   ('artes_e_artesanato', 'Artes e Artesanato'),
    ('esporte_lazer', 'Esporte e Lazer'),   ('la_cuisine', 'La Cuisine'),   ('informatica_acessorios', 'Informática e Acessórios'),
    ('consoles_games', 'Consoles e Games'), ('fashion_roupa_masculina', 'Fashion / Roupa Masculina'),
    ('moveis_colchao_e_estofado', 'Móveis / Colchão e Estofado'),   ('fraldas_higiene', 'Fraldas e Higiene'),
    ('artigos_de_festas', 'Artigos de Festa'),  ('tablets_impressao_imagem', 'Tablets, Impressão e Imagem'),
    ('ferramentas_jardim', 'Ferramentas / Jardim'), ('market_place', 'Marketplace'),
    ('seguros_e_servicos', 'Seguros e Serviços'),   ('moveis_escritorio', 'Móveis / Escritório'),
    ('cama_mesa_banho', 'Cama, Mesa e Banho'),  ('malas_acessorios', 'Malas e Acessórios'),
    ('bebes', 'Bebês'), ('eletroportateis', 'Eletroportáteis'), 
    ('construcao_ferramentas_construcao', 'Construção e Ferramentas / Construção'),
    ('fashion_underwear_e_moda_praia', 'Fashion / Underwear e Moda Praia'),
    ('papelaria', 'Papelaria'), ('moveis_decoracao', 'Móveis / Decoração'), ('cool_stuff', 'Cool Stuff'),
    ('eletrodomesticos', 'Eletrodomésticos'),   ('musica', 'Música'),
    ('instrumentos_musicais', 'Instrumentos Musicais'),
    ('moveis_cozinha_area_de_servico_jantar_e_jardim', 'Móveis / Cozinha, Área de Serviço, Jantar e Jardim'),
    ('casa_conforto', 'Casa e Conforto'),   ('casa_construcao', 'Casa e Construção')    

(71 rows affected)

Total execution time: 00:00:00.168

Por fim, adicionamos uma coluna à tabela de produtos que vai armazenar a versão reescrita do nome da categoria de cada produto:

In [50]:
ALTER TABLE olist_products ADD new_product_category_name NVARCHAR(255);

Commands completed successfully.

Total execution time: 00:00:00.144

E, finalmente, preenchemos a coluna com as categorias reescritas:

In [51]:
UPDATE olist_products
SET new_product_category_name = (SELECT pretty_name FROM #map_product_category_names WHERE ugly_name = olist_products.product_category_name)
WHERE EXISTS (SELECT pretty_name FROM #map_product_category_names WHERE ugly_name = olist_products.product_category_name)

(32309 rows affected)

Total execution time: 00:00:25.317

Feitas estas alterações, podemos retonar uma amostra dos registros que atendem aos critérios da consulta para conferir que o resultado final é o esperado:

In [55]:
SELECT TOP(20)
    product_id
    , product_category_name
    , new_product_category_name
    , product_height_cm
    , product_name_length
FROM olist_products
WHERE 
    product_height_cm >= 20 
    AND product_name_length BETWEEN 10 and 200;

(20 rows affected)

Total execution time: 00:00:00.210

product_id,product_category_name,new_product_category_name,product_height_cm,product_name_length
732bd381ad09e530fe0a5f457d81becb,cool_stuff,Cool Stuff,24,56
1c1890ba1779090cd54008a3c3302921,moveis_decoracao,Móveis / Decoração,24,27
f6574524b07d5bb753bab6f35abe2d06,malas_acessorios,Malas e Acessórios,25,52
051b9ff13dd55c0a6655a15ff296f80d,esporte_lazer,Esporte e Lazer,23,50
1eba879220bd0981a0e2fbab499ed4e1,moveis_escritorio,Móveis / Escritório,51,57
5370b82a213393979691c98074265584,bebes,Bebês,30,52
e6af694343b45b56304ad91974a110b9,utilidades_domesticas,Utilidades Domésticas,20,37
680874c570dad71c0a2844cfbf417054,moveis_decoracao,Móveis / Decoração,25,47
7a8dac4aaa16bc642e4df33adcf03303,cool_stuff,Cool Stuff,40,38
83b9bc6aae6f527ff6aafb9e01d6cbf3,brinquedos,Brinquedos,32,20


In [116]:
-- Finalmente, faço a alteração proposta no enunciado, mapeando categorias originais da
-- tabela olist_products_dataset para as categorias reescritas da tabela map_product_category_names:
UPDATE olist_products 
SET product_category_name = (SELECT better_name FROM #map_product_names WHERE ugly_name=olist_products.product_category_name)
WHERE EXISTS (SELECT better_name FROM #map_product_names WHERE ugly_name=olist_products.product_category_name);

(32328 rows affected)

Total execution time: 00:00:14.882

In [117]:
-- Por fim, exibo a tabela olist_products_dataset para confirmar que as alterações foram feitas
SELECT * FROM olist_products
WHERE product_height_cm >= 20 AND product_name_lenght BETWEEN 10 and 200;

(10050 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:02.962

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
732bd381ad09e530fe0a5f457d81becb,Cool / Stuff,56,1272,4,18350,70,24,44
1c1890ba1779090cd54008a3c3302921,Moveis / Decoracao,27,158,4,2550,29,24,45
f6574524b07d5bb753bab6f35abe2d06,Malas / Acessorios,52,573,5,3600,35,25,50
051b9ff13dd55c0a6655a15ff296f80d,Esporte / Lazer,50,529,2,3600,62,23,23
1eba879220bd0981a0e2fbab499ed4e1,Moveis / Escritorio,57,476,1,8950,52,51,17
5370b82a213393979691c98074265584,Bebes,52,708,9,7950,33,30,44
e6af694343b45b56304ad91974a110b9,Utilidades Domesticas,37,352,1,3900,19,20,15
680874c570dad71c0a2844cfbf417054,Moveis / Decoracao,47,502,1,600,25,25,25
7a8dac4aaa16bc642e4df33adcf03303,Cool / Stuff,38,1701,2,11750,18,40,79
83b9bc6aae6f527ff6aafb9e01d6cbf3,Brinquedos,20,866,3,2550,22,32,38


<br>
<hr>
<br>

## **Exercícios**

1. **Crie uma query que retorne os valores distintos de cidade existentes na tabela de clientes:**

In [14]:
-- Obs: a query a seguir 
-- (1) retorna as colunas listadas no SELECT, 
-- (2) ordena pelo critério do ORDER BY, 
-- (3) remove as duplicatas apontadas pelo DISTINCT e 
-- (4) limita o resultado à quantidade indicada pelo operador TOP
SELECT 
    DISTINCT TOP(20)    UPPER(customer_city) AS cidade, 
                        customer_state AS estado
FROM dbo.olist_customers
ORDER BY cidade, estado;

(20 rows affected)

Total execution time: 00:00:01.171

cidade,estado
ABADIA DOS DOURADOS,MG
ABADIANIA,GO
ABAETE,MG
ABAETETUBA,PA
ABAIARA,CE
ABAIRA,BA
ABARE,BA
ABATIA,PR
ABDON BATISTA,SC
ABELARDO LUZ,SC


2. **Crie uma query que retorne os valores distintos de cidade e estado, para os estados de São Paulo, Minas Gerais e Rio de Janeiro da tabela de clientes:**

In [16]:
-- A query é praticamente a mesma do item anterior, com o acréscimento da cláusula WHERE
SELECT 
    DISTINCT TOP(20)    UPPER(customer_city) AS cidade, 
                        customer_state AS estado
FROM dbo.olist_customers
WHERE customer_state IN ('SP', 'MG', 'RJ')
ORDER BY cidade, estado;

(20 rows affected)

Total execution time: 00:00:00.841

cidade,estado
ABADIA DOS DOURADOS,MG
ABAETE,MG
ABRE CAMPO,MG
ACAIACA,MG
ACUCENA,MG
ADAMANTINA,SP
ADOLFO,SP
AGISSE,SP
AGUA COMPRIDA,MG
AGUAI,SP


3. **Crie uma ou mais queries que retornem o preço, o frete, a data limite para envio e o identificador do pedido para os registros que têm o preço entre 50,00 e 250,00 e que têm, ao mesmo tempo, a data de envio limite maior do que 08 de Fevereiro de 2018. Utilize a tabela _olist\_order\_items:_**

In [41]:
SELECT TOP(30)
	order_id AS id_pedido
	, price AS preco
    , freight_value AS frete
    , shipping_limit_date AS data_limite_envio
FROM olist_order_items
WHERE 
	price BETWEEN 50 AND 250
    AND CONVERT(DATE, shipping_limit_date) > CONVERT(DATE, '2018-02-08')
ORDER BY data_limite_envio ASC, preco ASC;

(30 rows affected)

Total execution time: 00:00:00.995

id_pedido,preco,frete,data_limite_envio
83f0c4c6b1a3c31214ca8266dc888258,79.97,10.86,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.97,10.86,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.97,10.86,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.99,28.96,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.99,28.96,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.99,28.96,2018-02-09 00:18:24.0000000
83f0c4c6b1a3c31214ca8266dc888258,79.99,28.96,2018-02-09 00:18:24.0000000
b237e3e48b008b3ee341df062216e47d,59.00,12.75,2018-02-09 00:30:36.0000000
b237e3e48b008b3ee341df062216e47d,59.00,12.75,2018-02-09 00:30:36.0000000
6916bfc2979a20759a964f5addee58d3,139.99,28.34,2018-02-09 00:31:46.0000000


4. **Crie uma ou mais queries que retornem o preço, o frete, a data limite para envio e o identificador do pedido para os registros que têm o preço do frete inferior a 149,00 ou que têm um preço entre 250,00 e 500,00. Utilize a tabela *olist_order_items_dataset*:**

In [42]:
SELECT TOP(30)
	order_id AS identificador_pedido
	, price AS preco
    , freight_value AS frete
    , shipping_limit_date AS data_limite_envio
FROM olist_order_items
WHERE 
	freight_value < 149
    OR price BETWEEN 250 AND 500
ORDER BY data_limite_envio ASC, preco ASC, frete ASC;

(30 rows affected)

Total execution time: 00:00:12.831

identificador_pedido,preco,frete,data_limite_envio
e5fa5a7210941f7d56d0208e4e071d35,59.50,15.56,2016-09-19 00:15:34.0000000
bfbd0f9bdef84302105ad712db648a6c,44.99,2.83,2016-09-19 23:11:33.0000000
bfbd0f9bdef84302105ad712db648a6c,44.99,2.83,2016-09-19 23:11:33.0000000
bfbd0f9bdef84302105ad712db648a6c,44.99,2.83,2016-09-19 23:11:33.0000000
cd3b8574c82b42fc8129f6d502690c3e,29.99,10.96,2016-10-08 10:34:01.0000000
c3d9e402b6a0fbe2a5f7fc5b41117c38,189.00,48.45,2016-10-08 10:45:33.0000000
c4b41c36dd589e901f6879f25a74ec1d,9.90,8.72,2016-10-08 13:26:12.0000000
36989eb07a0de2d3d3129eea35553875,23.90,26.82,2016-10-08 13:46:32.0000000
36989eb07a0de2d3d3129eea35553875,23.90,26.82,2016-10-08 13:46:32.0000000
63638a6806d67773f3adba8534553fff,67.90,18.98,2016-10-08 13:47:45.0000000


5. **Crie uma query em SQL que retorne todos os tipos de pagamento. Utilize a tabela *olist_order_payments_dataset*:**

In [43]:
SELECT DISTINCT payment_type AS tipo_pagamento 
FROM olist_order_payments;

(5 rows affected)

Total execution time: 00:00:04.624

tipo_pagamento
credit_card
not_defined
debit_card
boleto
voucher


6. **Crie uma query que retorne o tipo e o valor do pagamento para as compras que foram parceladas de 12 a 24 vezes e que tiveram um valor superior a 245,99. Utilize a tabela *olist_order_payments_dataset*:**

In [45]:
SELECT TOP(20)
	order_id AS id_pedido
	, payment_type AS tipo_pagamento
    , payment_value AS valor_pagamento
    , payment_installments AS qtd_parcelas
FROM olist_order_payments 
WHERE 
	payment_installments between 12 and 24
    AND payment_value > 245.99
ORDER BY qtd_parcelas DESC;

(20 rows affected)

Total execution time: 00:00:00.459

id_pedido,tipo_pagamento,valor_pagamento,qtd_parcelas
70b7e94ea46d3e8b5bc12a50186edaf0,credit_card,274.84,24
859f516f2fc3f95772e63c5757ab0d5b,credit_card,609.56,24
ff36cbc44b8f228e0449c92ef089c843,credit_card,756.49,24
2b7dbe9be72b8f9733844c31055c0825,credit_card,345.39,24
6ae2e8b8fac02522481d2a2f4ca4412c,credit_card,433.43,24
90f864fe19d11549fa01eb81c4dd87e3,credit_card,588.58,24
84d2098c97827c6327ed4d7be95e1fc8,credit_card,286.78,24
ffb18bf111fa70edf316eb0390427986,credit_card,617.24,24
63dbe0c8e63e5f1b4deec09d4f044a7f,credit_card,771.69,24
fcbb6af360b31b05460c2c8e524588c0,credit_card,1194.38,24


7. **Crie uma query em SQL que retorne todas as pontuações de avaliação. Utilize a tabela *olist_order_reviews_dataset*:**

In [47]:
SELECT TOP(20)
	order_id AS id_pedido
    , review_score AS score_avaliacao 
from olist_order_reviews;
-- aqui talvez fizesse mais sentido uma consulta que contasse a quantidade de avaliações por nota, como:
-- SELECT review_score AS score_avaliacao, COUNT() AS quantidade FROM olist_order_reviews GROUP BY review_score;

(20 rows affected)

Total execution time: 00:00:00.152

id_pedido,score_avaliacao
73fc7af87114b39712e6da79b0a377eb,4
a548910a1c6147796b98fdf73dbeba33,5
f9e4b658b201a9f2ecdecbb34bed034b,5
658677c97b385a9be170737859d3511b,5
8e6bfb81e283fa7e4f11123a3fb894f1,5
b18dcdf73be66366873cd26c5724d1dc,1
e48aa0d2dcec3a2e87348811bcfdf22b,5
c31a859e34e3adac22f376954e19b39d,5
9c214ac970e84273583ab523dfafd09b,5
b9bf720beb4ab3728760088589c62129,4


8. **Crie uma query em SQL que retorne todos os status de pedidos. Utilize a tabela *olist_orders_dataset*:**

In [49]:
SELECT top(20)
	order_id AS id_pedido
    , order_status AS status_pedido 
FROM olist_orders;

(20 rows affected)

Total execution time: 00:00:00.203

id_pedido,status_pedido
e481f51cbdc54678b7cc49136f2d6af7,delivered
53cdb2fc8bc7dce0b6741e2150273451,delivered
47770eb9100c2d0c44946d9cf07ec65d,delivered
949d5b44dbf5de918fe9c16f97b45f8a,delivered
ad21c59c0840e6cb83a9ceb5573f8159,delivered
a4591c265e18cb1dcee52889e2d8acc3,delivered
136cce7faa42fdb2cefd53fdc79a6098,invoiced
6514b8ad8028c9f2cc2374ded245783f,delivered
76c6e866289321a7c93b82b54852dc33,delivered
e69bfb5eb88e0ed6a785585b27e16dbf,delivered


9. **Crie uma query que delete os registros dos pedidos que tenham o status igual a *unavailable* e que têm data de aprovação igual ou anterior a 10 de Outubro de 2017. Utilize a tabela *olist_orders_dataset*:**

In [50]:
DELETE from olist_orders
WHERE order_id IN (
  SELECT order_id 
  FROM olist_orders 
  WHERE order_status = 'unavailable' AND CONVERT(DATE, order_approved_at) <= CONVERT(DATE, '2017-10-10')
);

(294 rows affected)

Total execution time: 00:00:04.909